In [4]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd


/Users/cam/Documents/diss/copy/synoptic-project-CameronFMacKay/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
model = SentenceTransformer("all-MiniLM-L6-v2")  # Small, fast & effective


In [7]:
df = pd.read_csv('songs_lyrics_adjusted.csv')
df["lyrics"] = df["lyrics"].fillna("")

In [8]:
df.head()

,title,tag,artist,year,views,features,lyrics,id,language_cld3,language_ft,language
0,Killa Cam,rap,Cam'ron,2004,173166,"{""Cam\\'ron"",""Opera Steve""}",chorus opera steve camron\nkilla cam killa ca...,1,en,en,en
1,Can I Live,rap,JAY-Z,1996,468624,{},produced by irv gotti\n\nintro\nyeah hah yeah ...,3,en,en,en
2,Forgive Me Father,rap,Fabolous,2003,4743,{},maybe cause im eatin\nand these bastards fiend...,4,en,en,en
3,Down and Out,rap,Cam'ron,2004,144404,"{""Cam\\'ron"",""Kanye West"",""Syleena Johnson""}",produced by kanye west and brian miller\n\nint...,5,en,en,en
4,Fly In,rap,Lil Wayne,2005,78271,{},intro\nso they ask me\nyoung boy\nwhat you gon...,6,en,en,en


In [9]:
newdf = df[["title", "lyrics"]]  # Keep necessary info

testdf = newdf[0:1000]


In [13]:
import torch

# Set device to MPS if available, otherwise use CPU
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

# Example image tag from classified image
image_tag = "bright airport airplane"

# Generate image tag embedding and move it to MPS
image_tag_embedding = model.encode(image_tag, convert_to_tensor=True).to(device)


In [17]:

testdf.loc[:, "song_embedding"] = testdf.apply(
    lambda row: model.encode(
        f"{row['title']} {row['lyrics']}", convert_to_tensor=True
    ).to(torch.device("mps")).cpu().numpy(),  # Move tensor to MPS, then to CPU for NumPy conversion
    axis=1
)


In [24]:
print(len(testdf['song_embedding'].iloc[44]))

384


In [18]:
# Compute similarity between image tag and each song


testdf.loc[:,"similarity"] = testdf["song_embedding"].apply(
    lambda emb: util.pytorch_cos_sim(image_tag_embedding, torch.tensor(emb, device=image_tag_embedding.device)).item()
)


# Find the best-matching song
best_match = testdf.loc[testdf["similarity"].idxmax()]
best_match_index = testdf["similarity"].idxmax()  # Get index of the best match
best_match = testdf.loc[best_match_index]  # Get the best match row

# Retrieve the artist from the original df using the same position
artist_name = df.iloc[best_match_index]["artist"]


# Print result
print(f"Best-matching song: {best_match['title']} by {artist_name} (Score: {best_match['similarity']:.4f})")


Best-matching song: Airplanes by B.o.B (Score: 0.3747)
